## 载入数据

我们仅仅使用 1997 年以后的数据集：

In [1]:
import pandas as pd
import numpy as np

In [2]:
gtd_df = pd.read_hdf('./data/gtd_1998-2017.h5')  # 载入数据集

In [3]:
import json

with open('./data/gtd_1998-2017_names.json') as fp:
    names = json.load(fp)                           # 载入变量名

In [23]:
class Bunch(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__dict__ = self

In [24]:
names = Bunch(names)

下面我们看看 GTD 的数据:

In [35]:
gtd_df.head(2)

,eventid,iyear,imonth,iday,extended,country,country_txt,region,region_txt,provstate,...,ishostkid,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
67507,199801010001,1998,1,1,0,34,Burundi,11,Sub-Saharan Africa,Bujumbura Mairie,...,0.0,Unknown,"Burundi Rebels, Ex-Rwandan Army Soldiers Blam...",Burundi--Attack Reported on Bujumbura Airport...,Unknown,CETIS,0.0,1.0,0.0,1.0
67508,199801010002,1998,1,1,0,167,Russia,9,Eastern Europe,Moscow (Federal City),...,0.0,Unknown,"Bomb injures 3 in Moscow subway system, The ...","Bomb injures 3 in Moscow subway, Charleston ...","Bomb Injures 3 Workers in Moscow Metro, Los ...",CETIS,-1.0,-1.0,0.0,-1.0


In [6]:
print('数据的尺寸：', gtd_df.shape)

数据的尺寸： (114184, 65)


通过前面的数据预处理我们已经将文本变量的缺失值替换为 `Unknown`，分类变量的缺失值替换为 `-1`，但是对于数值型变量的缺失值未做填充处理。这个填充工作不能大而化之，需要针对具体问题具体分析。为了下面我们讨论的方便，我们先将所有含缺失值的样本均丢掉：

In [10]:
df = gtd_df.dropna()

np.unique(df.isnull().sum())  # 不含缺失值

array([0], dtype=int64)

In [11]:
df.shape

(99521, 65)

In [21]:
miss_df = gtd_df.loc[list(set(gtd_df.index) - set(df.index)),:]  # 含有缺失值的样本集

df.shape[0] + miss_df.shape[0] == gtd_df.shape[0]

True

下面主要研究 `df` 数据集：

对于一个机器学习问题，首先需要做的的工作是**识别或确定问题**。

## 第一步：确定问题

我们首先要明确要研究的问题是**分类**还是**回归**问题？一般数据由特征 $X$ 和标签 $y$ 来构成，$y$ 可以是一列也可以是多列，可以是二值型 (布尔值型数据类型，一般使用 `0` 和 `1` 进行编码) 也可以是实数型。一般地，它为二值型时该问题为分类问题，它为实数型是该问题为回归问题。

## 第二步：分离数据

一般为了防止模型出现过拟合现象，需要将数划分为**训练集**和**验证集**。即使用训练集训练模型，使用验证集调节超参数。

In [22]:
from sklearn.model_selection import StratifiedKFold  # 分类问题
#from sklearn.model_selection import KFold        # 回归问题

In [28]:
names.NV  # 查看数值变量

['nkillter',
 'imonth',
 'nperpcap',
 'nkillus',
 'nkill',
 'iday',
 'nwoundus',
 'nwoundte',
 'latitude',
 'iyear',
 'nwound',
 'longitude',
 'eventid']

这里我们仅仅选择以下几个数值变量:

In [31]:
nv = [
    'nkillter','nperpcap','nkillus','nkill','nwoundus','nwoundte', 'nwound',
]

X = df[nv]

取 `'success'` 当作标签向量，即：

In [34]:
y = df['success']

In [41]:
np.unique(y)

array([0, 1], dtype=int64)

`y` 是一个二值型变量，该问题是一个二分类问题。

划分数据集：

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25,random_state = 7)

下面查看数据的标签分布：

In [46]:
y_val.value_counts()

1    21526
0     3355
Name: success, dtype: int64

In [47]:
y_train.value_counts()

1    64471
0    10169
Name: success, dtype: int64

对数值变量进行标准化处理:

In [51]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)

使用逻辑回归模型进行分类

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
lr = LogisticRegression()
sgdc = SGDClassifier(max_iter = 100)
lr.fit(X_train, y_train)  # 训练模型
lr_y_predict = lr.predict(X_val)   # 预测

使用 SGDClassifier 训练模型参数

In [94]:
%%time
sgdc.fit(X_train, y_train) # 训练
sgdc_y_predict = sgdc.predict(X_val)   # 预测

Wall time: 626 ms


### 性能评估

In [59]:
from sklearn.metrics import classification_report

In [66]:
print('LR 分类器的准确率：', lr.score(X_val,y_val))

print(classification_report(y_val, lr_y_predict, target_names=['success', 'no success']))

LR 分类器的准确率： 0.8658414050882199
             precision    recall  f1-score   support

    success       0.72      0.01      0.02      3355
 no success       0.87      1.00      0.93     21526

avg / total       0.85      0.87      0.81     24881



In [95]:
print('SGD 分类器的准确率：', sgdc.score(X_val,y_val))

print(classification_report(y_val, sgdc_y_predict, target_names=['success', 'no success']))

SGD 分类器的准确率： 0.8651581528073631
             precision    recall  f1-score   support

    success       0.00      0.00      0.00      3355
 no success       0.87      1.00      0.93     21526

avg / total       0.75      0.87      0.80     24881



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
